<a href="https://colab.research.google.com/github/jovanape/Bitcoint-Price-Prediction/blob/main/Utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fajl sa pomoćnim funkcijama**

### **Učitavanje biblioteka**

In [ ]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
#from keras.backend.tensorflow_backend import set_session
#from keras.callbacks import CSVLogger, ModelCheckpoint
#from keras.models import Sequential, Dense, Dropout, LSTM
#import h5py
import os
from keras.losses import MeanSquaredError

In [ ]:
DAYS_TO_PREDICT = 60

### **Podela skupa podataka**

In [ ]:
# Definicija funkcije za podelu podataka na train i test skup

def data_split(data, test_size = 0):

  if test_size <= 0:
    print('Vrednost parametra test_size mora biti strogo veca od 0.\n')
    return
  else: # test_size > 0
    test_limit = len(data) - int(test_size * len(data))

  train = data[:test_limit]
  test = data[test_limit:]
  
  return train, test

### **Vizualizacija podataka**

In [ ]:
# Definicija funkcije za vizuelni prikaz train i test podataka

def plot_data(train_data = train['Close'], test_data = test['Close']):

  fig, ax = plt.subplots(1, figsize=(16, 9))
  ax.plot(train_data, label='trening', linewidth=2)
  ax.plot(test_data, label='test', linewidth=2)
  ax.set_ylabel('cena [USD]', fontsize=14)
  ax.set_title(label = 'Vizuelni prikaz skupova za trening i test', fontsize=18)
  ax.legend(loc='best', fontsize=18)

  #data = data.reset_index()
  #x = data.index
  #ax.set_xticks(range(87724))

### **Filtriranje atributa datog skupa podataka**

In [ ]:
# Izbacivanje nepotrebnih kolona (atributa) iz skupa dataset_

def filter_data_set(dataset_):
  new_dataset_ = dataset_.drop(['Open', 'High', 'Low', 'Date', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Timestamp'], axis = 1)
  print('Filtriran skup:\n', new_dataset_.head())
  print('\nBroj atributa pre i nakon izbacivanja:\n', dataset_.shape[1], new_dataset_.shape[1])
  return new_dataset_

### **Skaliranje trening skupa**

In [ ]:
def scale_train(train_dataset):
  scaler = MinMaxScaler(feature_range=(0, 1))
  train_dataset = scaler.fit_transform(train_dataset)
  return scaler, train_dataset

### **Vizuelni prikaz trening i validacionog skupa**

In [ ]:
def plot_train_and_validation_data(train, validation):
  plt.title('Podaci za trening i validaciju')
  plt.xlabel('vreme')
  plt.ylabel('vrednost bitkoina')
  plt.plot(train, c = 'lime')
  plt.plot(validation, c = 'orchid')
  plt.show()

### **Kompilacija i treniranje mreze**

In [ ]:
# Definicija funkcije za kompilaciju i treniranje mreze
# Treniranje je moguce vrsiti sa ili bez validacije

def compile_and_fit(net, optimizer, loss, metrics, X_train, y_train, epochs, batch_size, validation = 1, validation_data = None, patience = 3):

  net.compile(optimizer = optimizer, loss = loss, metrics = metrics)

  callback = callbacks.EarlyStopping(monitor = 'loss', patience = patience)

  if validation == 0: # bez validacije
    history = net.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, callbacks=[callback])
  else: # sa validacijom
    history = net.fit(X_train, y_train, epochs = epochs, validation_data = validation_data, batch_size = batch_size, callbacks=[callback])

  return history, net

### **Vizualizacija rezultata predvidjanja**

In [ ]:
# Vizuelni prikaz rezultata modela mreze

#def plot_real_vs_predicted_data(test_set, predicted_set, caption):
#
#  fig, ax = plt.subplots(1, figsize=(16, 9))
#  ax.plot(test_set, label='Stvarna vrednost BTC', linewidth=2)
#  ax.plot(predicted_set, label='Predvidjena vrednost BTC', linewidth=2)
#  ax.set_xlabel('Timestamp', fontsize=14)
#  ax.set_ylabel('Cena [USD]', fontsize=14)
#  ax.set_title(label = caption, fontsize=18)
#  ax.legend(loc='best', fontsize=18)

  # test = test.reset_index()
  # x = test.index


def plot_real_vs_predicted_data(test_set, predicted_set):
  
  plt.plot(test_set, label='Stvarna vrednost BTC', linewidth=2)
  plt.plot(predicted_set, label='Predvidjena vrednost BTC', linewidth=2)
  plt.xlabel('Redni broj instance', fontsize=14)
  plt.ylabel('Cena [USD]', fontsize=14)
  plt.title(label = 'Vizuelni prikaz stvarne i predvidjene vrednosti BTC', fontsize=18)
  plt.legend(loc='best', fontsize=18)
  plt.show()

  # test = test.reset_index()
  # x = test.index

### Vizuelni prikaz funkcije gubitka, MSE i MAE tokom treniranja mreze

In [ ]:
# Vizuelni prikaz funkcije gubitka

def plot_loss(history_):

  epochs = range(len(history_.history['loss'])) # broj epoha

  plt.title('Vizuelni prikaz gubitka (loss) i MSE (mse) mreze')

  plt.subplot(211)
  
  plt.plot(epochs, history_.history['loss'], label='Gubitak (loss) tokom treniranja')

  if 'val_loss' in history_.history:
    plt.plot(epochs, history_.history['val_loss'], label='Gubitak (val_loss) tokom validacije')
  
  plt.legend()


# Vizuelni prikaz MSE

def plot_mse(history_):

  epochs = range(len(history_.history['mse'])) # broj epoha

  plt.title('Vizuelni prikaz srednjekvadratne greske (MSE) mreze')

  plt.subplot(212)

  plt.plot(epochs, history_.history['mse'], label='MSE (mse) tokom treniranja')
  
  if 'val_mse' in history_.history:
    plt.plot(epochs, history_.history['val_mse'], label='MSE (val_mse) tokom validacije')
  
  plt.legend()

  plt.show()


# Vizuelni prikaz MAE

def plot_mae(history_):

  epochs = range(len(history_.history['mae'])) # broj epoha
  plt.title('Vizuelni prikaz MAE (mae')
  plt.xlabel('broj epohe')
  plt.ylabel('MAE')
  
  plt.plot(epochs, history_.history['mae'], label='MAE (mae) tokom treniranja')

  if 'val_mae' in history_.history:
    plt.plot(epochs, history_.history['val_mae'], label='MAE (val_mae) tokom treniranja sa validacijom')

  plt.legend(loc = 'best')

  plt.show()